In [3]:
import wandb
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist

In [2]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vinyk-sd (vinyk-sd-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
# Load dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize and reshape data
X_train, X_test = X_train / 255.0, X_test / 255.0
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# One-hot encoding
y_train_one_hot = np.eye(10)[y_train]
y_test_one_hot = np.eye(10)[y_test]

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [5]:
class NeuralNetwork:
    def __init__(self, layers, activation=None, output_activation = 'softmax', weight_init=None):
        """
        Initializes the neural network.
        :param layers: List representing the number of neurons in each layer.
        :param activation: Activation function for hidden layers ('relu', 'sigmoid', 'tanh').
        :param output_activation: Activation function for the output layer ('softmax', 'sigmoid', or 'linear').
        :param weight_init: Weight initialization method ('random', 'xavier').
        """
        self.layers = layers
        self.activation = activation
        self.output_activation = output_activation  # Separate activation for output layer
        self.weight_init = weight_init
        self.weights, self.biases = self._initialize_weights()

    def _initialize_weights(self):
        """Initializes weights and biases based on the selected method."""
        np.random.seed(42)
        weights, biases = [], []

        for i in range(len(self.layers) - 1):
            if self.weight_init == "xavier":
                limit = np.sqrt(6 / (self.layers[i] + self.layers[i+1]))
                w = np.random.uniform(-limit, limit, (self.layers[i], self.layers[i+1]))
            else:  # Default: Random small values
                w = np.random.randn(self.layers[i], self.layers[i+1]) * 0.01
            
            b = np.zeros((1, self.layers[i+1]))
            weights.append(w)
            biases.append(b)
        
        return weights, biases

    def _activation(self, x, func, derivative=False):
        """Applies the selected activation function."""
        if func == "relu":
            return np.where(x > 0, 1, 0) if derivative else np.maximum(0, x)
        elif func == "tanh":
            t = np.tanh(x)
            return 1 - t**2 if derivative else t
        elif func == "sigmoid":
            sig = 1 / (1 + np.exp(-x))
            return sig * (1 - sig) if derivative else sig
        elif func == "softmax":
            exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Stability trick
            return exp_x / np.sum(exp_x, axis=1, keepdims=True)
        else:  # Linear (no activation)
            return x

    def forward(self, X):
        """Performs forward propagation."""
        activations, zs = [X], []

        for i, (w, b) in enumerate(zip(self.weights, self.biases)):
            z = np.dot(activations[-1], w) + b
            zs.append(z)

            # Use different activation for last layer
            if i == len(self.weights) - 1:
                activations.append(self._activation(z, self.output_activation))
            else:
                activations.append(self._activation(z, self.activation))
        
        return activations, zs

    def backward(self, y_true, activations, zs, learning_rate):
        """Performs backpropagation and computes gradients."""
        grads_w, grads_b = [], []
        m = y_true.shape[0]
        
        # Compute loss gradient (assuming MSE loss for now)
        dA = activations[-1] - y_true  

        for i in reversed(range(len(self.weights))):
            activation_func = self.output_activation if i == len(self.weights) - 1 else self.activation
            dz = dA * self._activation(zs[i], activation_func, derivative=True)
            dw = np.dot(activations[i].T, dz) / m
            db = np.sum(dz, axis=0, keepdims=True) / m
            dA = np.dot(dz, self.weights[i].T)

            grads_w.insert(0, dw)
            grads_b.insert(0, db)

        return grads_w, grads_b


In [6]:
# Optimizers
class Optimizer:
    def __init__(self, method= None, lr=None, beta=0.9, beta2=0.999, epsilon=1e-8):
        self.method = method
        self.lr = lr
        self.beta = beta  # Used for momentum-based optimizers
        self.beta2 = beta2  # Used for RMSprop, Adam, Nadam
        self.epsilon = epsilon  # Prevents division by zero
        self.m_w = None  # First moment vector for weights
        self.v_w = None  # Second moment vector for weights
        self.m_b = None  # First moment vector for biases
        self.v_b = None  # Second moment vector for biases
        self.t = 0  # Time step for bias correction

    def update(self, weights, biases, grads_w, grads_b):
        if self.method == "sgd":
            weights = [w - self.lr * dw for w, dw in zip(weights, grads_w)]
            biases = [b - self.lr * db for b, db in zip(biases, grads_b)]

        elif self.method in ["momentum", "nesterov"]:
            if self.m_w is None:  # Initialize momentum terms
                self.m_w = [np.zeros_like(w) for w in weights]
                self.m_b = [np.zeros_like(b) for b in biases]

            for i in range(len(weights)):
                if self.method == "nesterov":
                    # Look-ahead step
                    look_ahead_w = weights[i] - self.beta * self.m_w[i]
                    look_ahead_b = biases[i] - self.beta * self.m_b[i]

                    # Compute gradients at look-ahead position
                    self.m_w[i] = self.beta * self.m_w[i] + self.lr * grads_w[i]
                    self.m_b[i] = self.beta * self.m_b[i] + self.lr * grads_b[i]

                    weights[i] = look_ahead_w - self.m_w[i]
                    biases[i] = look_ahead_b - self.m_b[i]

                else:  # Normal momentum
                    self.m_w[i] = self.beta * self.m_w[i] + self.lr * grads_w[i]
                    self.m_b[i] = self.beta * self.m_b[i] + self.lr * grads_b[i]
                    weights[i] -= self.m_w[i]
                    biases[i] -= self.m_b[i]

        elif self.method in ["rmsprop", "adam", "nadam"]:
            if self.m_w is None:  # Initialize moment estimates
                self.m_w = [np.zeros_like(w) for w in weights]
                self.v_w = [np.zeros_like(w) for w in weights]
                self.m_b = [np.zeros_like(b) for b in biases]
                self.v_b = [np.zeros_like(b) for b in biases]

            self.t += 1  # Update time step

            for i in range(len(weights)):
                if self.method == "rmsprop":
                    # Update second moment estimate (squared gradient)
                    self.v_w[i] = self.beta2 * self.v_w[i] + (1 - self.beta2) * grads_w[i] ** 2
                    self.v_b[i] = self.beta2 * self.v_b[i] + (1 - self.beta2) * grads_b[i] ** 2

                    # Update weights and biases
                    weights[i] -= self.lr * grads_w[i] / (np.sqrt(self.v_w[i]) + self.epsilon)
                    biases[i] -= self.lr * grads_b[i] / (np.sqrt(self.v_b[i]) + self.epsilon)

                elif self.method in ["adam", "nadam"]:
                    # Compute biased first moment estimate
                    self.m_w[i] = self.beta * self.m_w[i] + (1 - self.beta) * grads_w[i]
                    self.m_b[i] = self.beta * self.m_b[i] + (1 - self.beta) * grads_b[i]

                    # Compute biased second moment estimate
                    self.v_w[i] = self.beta2 * self.v_w[i] + (1 - self.beta2) * (grads_w[i] ** 2)
                    self.v_b[i] = self.beta2 * self.v_b[i] + (1 - self.beta2) * (grads_b[i] ** 2)

                    # Bias correction
                    m_hat_w = self.m_w[i] / (1 - self.beta ** self.t)
                    v_hat_w = self.v_w[i] / (1 - self.beta2 ** self.t)
                    m_hat_b = self.m_b[i] / (1 - self.beta ** self.t)
                    v_hat_b = self.v_b[i] / (1 - self.beta2 ** self.t)

                    if self.method == "adam":
                        # Adam update rule
                        weights[i] -= self.lr * m_hat_w / (np.sqrt(v_hat_w) + self.epsilon)
                        biases[i] -= self.lr * m_hat_b / (np.sqrt(v_hat_b) + self.epsilon)

                    elif self.method == "nadam":
                        # Nadam additional momentum correction
                        nadam_m_w = self.beta * m_hat_w + (1 - self.beta) * grads_w[i] / (1 - self.beta ** self.t)
                        nadam_m_b = self.beta * m_hat_b + (1 - self.beta) * grads_b[i] / (1 - self.beta ** self.t)

                        weights[i] -= self.lr * nadam_m_w / (np.sqrt(v_hat_w) + self.epsilon)
                        biases[i] -= self.lr * nadam_m_b / (np.sqrt(v_hat_b) + self.epsilon)

        return weights, biases

In [7]:
def train():
    wandb.init(project="Assignment1_Attempt3")
    config = wandb.config  # Get hyperparameters
    run_name = f"hl_{config.hidden_layers}_bs_{config.batch_size}_ac_{config.activation}_e_{config.epochs}"
    print(run_name)
    wandb.run.name = run_name

    # Extract hyperparameters from WandB
    layers = [784] + [config.hidden_size] * config.hidden_layers + [10]
    activation = config.activation
    weight_init = config.weight_init
    learning_rate = config.learning_rate
    batch_size = config.batch_size
    optimizer = config.optimizer
    epochs = config.epochs


    # Initialize Neural Network
    model = NeuralNetwork(layers, activation=activation, weight_init=weight_init)
    optimizer = Optimizer(method=optimizer, lr=learning_rate)

    for epoch in range(epochs):
        num_batches = len(X_train) // batch_size
        train_loss, train_acc = 0, 0

        for i in range(num_batches):
            start, end = i * batch_size, (i + 1) * batch_size
            X_batch, y_batch = X_train[start:end], y_train_one_hot[start:end]

            # Forward propagation
            activations, zs = model.forward(X_batch)
            preds = np.argmax(activations[-1], axis=1)
            y_true_labels = np.argmax(y_batch, axis=1)

            # Compute loss and accuracy
            batch_loss = np.mean((activations[-1] - y_batch) ** 2)
            batch_acc = np.mean(preds == y_true_labels)

            train_loss += batch_loss
            train_acc += batch_acc

            # Backpropagation
            grads_w, grads_b = model.backward(y_batch, activations, zs, learning_rate)
            model.weights, model.biases = optimizer.update(model.weights, model.biases, grads_w, grads_b)

        # Average loss and accuracy
        train_loss /= num_batches
        train_acc /= num_batches

        # Validation phase
        val_activations, _ = model.forward(X_test)
        val_preds = np.argmax(val_activations[-1], axis=1)
        val_true_labels = np.argmax(y_test_one_hot, axis=1)

        val_loss = np.mean((val_activations[-1] - y_test_one_hot) ** 2)
        val_acc = np.mean(val_preds == val_true_labels)

        # Log metrics to WandB
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "val_loss": val_loss,
            "train_accuracy": train_acc,
            "val_accuracy": val_acc
        })

        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")


In [8]:
sweep_config = {
    'method': 'bayes',  # Bayesian optimization for efficiency
    'metric': {'name': 'val_loss', 'goal': 'minimize'},  # Optimize for lowest validation loss
    'parameters': {
        'epochs': {'values': [5, 10, 15]},
        'hidden_layers': {'values': [3, 4, 5]},
        'hidden_size': {'values': [32, 64, 128]},
        'weight_decay': {'values': [0, 0.0005, 0.5]},
        'learning_rate': {'values': [1e-3, 1e-4]},
        'optimizer': {'values': ['sgd', 'momentum', 'nesterov', 'rmsprop', 'adam', 'nadam']},
        'batch_size': {'values': [16, 32, 64]},
        'weight_init': {'values': ['random', 'xavier']},
        'activation': {'values': ['sigmoid', 'tanh', 'relu']},  # <--- Added missing comma
        'epsilon': {'values': [0.8, 0.9, 0.95]}
    }
}

In [9]:
sweep_id = wandb.sweep(sweep_config, project="Assignment_Attempt3")
wandb.agent(sweep_id, train, count=20)  # Run 200 different hyperparameter combinations

Create sweep with ID: cs09wxtb
Sweep URL: https://wandb.ai/vinyk-sd-indian-institute-of-technology-madras/Assignment_Attempt3/sweeps/cs09wxtb


wandb: Agent Starting Run: id4x9yv6 with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 32
wandb: 	epochs: 15
wandb: 	epsilon: 0.9
wandb: 	hidden_layers: 4
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0.0005
wandb: 	weight_init: random


hl_4_bs_32_ac_sigmoid_e_15
Epoch 1: Train Loss=0.0872, Train Acc=0.1508, Val Loss=0.0833, Val Acc=0.2102
Epoch 2: Train Loss=0.0832, Train Acc=0.2084, Val Loss=0.0835, Val Acc=0.2105
Epoch 3: Train Loss=0.0832, Train Acc=0.2072, Val Loss=0.0820, Val Acc=0.2166
Epoch 4: Train Loss=0.0818, Train Acc=0.2095, Val Loss=0.0815, Val Acc=0.2235
Epoch 5: Train Loss=0.0812, Train Acc=0.2267, Val Loss=0.0804, Val Acc=0.2571
Epoch 6: Train Loss=0.0765, Train Acc=0.2828, Val Loss=0.0738, Val Acc=0.3001
Epoch 7: Train Loss=0.0730, Train Acc=0.3075, Val Loss=0.0718, Val Acc=0.3319
Epoch 8: Train Loss=0.0707, Train Acc=0.3562, Val Loss=0.0694, Val Acc=0.3766
Epoch 9: Train Loss=0.0685, Train Acc=0.3725, Val Loss=0.0677, Val Acc=0.3790
Epoch 10: Train Loss=0.0672, Train Acc=0.3734, Val Loss=0.0668, Val Acc=0.3752
Epoch 11: Train Loss=0.0665, Train Acc=0.3715, Val Loss=0.0663, Val Acc=0.3737
Epoch 12: Train Loss=0.0661, Train Acc=0.3705, Val Loss=0.0661, Val Acc=0.3702
Epoch 13: Train Loss=0.0655, Train

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_accuracy,▁▃▃▃▃▅▆▇███████
train_loss,█▇▇▆▆▅▄▃▂▂▂▂▂▁▁
val_accuracy,▁▁▁▂▃▅▆▇█▇▇▇▇██
val_loss,██▇▇▇▅▄▃▂▂▂▂▂▁▁
epoch,15
train_accuracy,0.38375
train_loss,0.06384
val_accuracy,0.3894
val_loss,0.06353


wandb: Agent Starting Run: taix6a4y with config:
wandb: 	activation: relu
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	epsilon: 0.8
wandb: 	hidden_layers: 5
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.001
wandb: 	optimizer: momentum
wandb: 	weight_decay: 0
wandb: 	weight_init: xavier


...

hl_3_bs_64_ac_tanh_e_5
Epoch 1: Train Loss=0.0586, Train Acc=0.5612, Val Loss=0.0402, Val Acc=0.7272
Epoch 2: Train Loss=0.0327, Train Acc=0.7788, Val Loss=0.0273, Val Acc=0.8138
Epoch 3: Train Loss=0.0237, Train Acc=0.8427, Val Loss=0.0204, Val Acc=0.8649
Epoch 4: Train Loss=0.0188, Train Acc=0.8738, Val Loss=0.0177, Val Acc=0.8820
Epoch 5: Train Loss=0.0165, Train Acc=0.8894, Val Loss=0.0158, Val Acc=0.8957


epoch,▁▃▅▆█
train_accuracy,▁▆▇██
train_loss,█▄▂▁▁
val_accuracy,▁▅▇▇█
val_loss,█▄▂▂▁
epoch,5
train_accuracy,0.88937
train_loss,0.01648
val_accuracy,0.8957
val_loss,0.01585


wandb: Agent Starting Run: oy52tdvk with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	epsilon: 0.95
wandb: 	hidden_layers: 4
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: nadam
wandb: 	weight_decay: 0
wandb: 	weight_init: random


hl_4_bs_32_ac_sigmoid_e_5
Epoch 1: Train Loss=0.0883, Train Acc=0.1383, Val Loss=0.0822, Val Acc=0.2086
Epoch 2: Train Loss=0.0816, Train Acc=0.2183, Val Loss=0.0812, Val Acc=0.2253
Epoch 3: Train Loss=0.0771, Train Acc=0.3072, Val Loss=0.0749, Val Acc=0.3262
Epoch 4: Train Loss=0.0781, Train Acc=0.2730, Val Loss=0.0804, Val Acc=0.3032
Epoch 5: Train Loss=0.0800, Train Acc=0.2547, Val Loss=0.0791, Val Acc=0.2098


epoch,▁▃▅▆█
train_accuracy,▁▄█▇▆
train_loss,█▄▁▂▃
val_accuracy,▁▂█▇▁
val_loss,█▇▁▆▅
epoch,5
train_accuracy,0.25467
train_loss,0.07996
val_accuracy,0.2098
val_loss,0.07912
